## Statistical Analysis of GReatModelS simulations 

In this notebook I have used spark (pyspark) to perform the statistical analysis of the different climatic variables. The main objective is to compare the different simulations. First we find the differential analysis of the continuous simulation versus the distributed one and then the analysis corresponding to the different soil models with a use of the same soil.

In [1]:
#Initializating Spark Session
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("stats") \
                    .appName('stats') \
                    .getOrCreate()

# https://stackoverflow.com/questions/44135610/spark-scala-split-dataframe-into-equal-number-of-rows

In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

header = re.compile(r"^\s+(\d{10})\s+(\d*)\s+(\d*)\s+(\d*)$")

df = pd.DataFrame()

with open("t2.dat", "r") as ifp:
    rows = []
    date = None
    count = 0
    
    while line := ifp.readline():
        # Get the header and start a new file
        if m := header.match(line):
            # We have a header so convert to array then flatten to a vector
            # before appending to the dataframe.
            if rows and date:
                df[date] = np.array(rows, dtype=float).flatten(order="C")
                rows = []
                
            # Get the header
            date = m.group(1)
            
        else:
            rows.append(line.strip().split())
    
    print(f"Appending the last {len(rows)*len(rows[0])} values")
    df[date] = np.array(rows, dtype=float).flatten(order="C")


In [ ]:
import re
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window


df = spark.read.text("t2.dat")

headers = df.withColumn("header", regexp_extract(col("value"), r"^\s+(\d{10})\s+(\d*)\s+(\d*)\s+(\d*)$", 1)).select("header")

def blank_as_null(x):
    return when(col(x) != "", col(x)).otherwise(None)


dates = headers.withColumn("dates", blank_as_null("header")).select("dates").dropna()

df1= df.withColumn("lol",explode(split(col("value"),'  '))).\
drop("genre1").\
drop("value").\
show()

In [4]:
#w = Window.rowsBetween(Window.unboundedPreceding, 0)

#df1 = df.withColumn('tmp', when(df.value.startswith('^\s+(\d{10})\s+(\d*)\s+(\d*)\s+(\d*)$'), df.value).otherwise(None)).withColumn('temp', last('tmp', True).over(w)).drop('tmp')

#df2 = df1.filter(df1.value != df1.temp).groupBy(df1.temp).agg(concat_ws(',',collect_list(df1.value)).alias('column2')).withC

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/sergiolp/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sergiolp/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/sergiolp/spark-3.1.1-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/sergiolp/spark-3.1.1-bin-hadoop2.7/python/pyspark/sql/dataframe.py", line 677, 

Py4JError: An error occurred while calling o18.setCallSite